<a href="https://colab.research.google.com/github/yzhou359/MakeItTalk/blob/main/quick_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MakeItTalk Quick Demo (natural human face animation)

- included project setup + pretrained model download
- provides step-by-step details
- todo: tdlr version

In [1]:
import sys
sys.path.append("thirdparty/AdaptiveWingLoss")
import os, glob
import numpy as np
import cv2
import argparse
from src.approaches.train_image_translation import Image_translation_block
import torch
import pickle
import face_alignment
from src.autovc.AutoVC_mel_Convertor_retrain_version import AutoVC_mel_Convertor
import shutil
import time
import util.utils as util
from scipy.signal import savgol_filter
from src.approaches.train_audio2landmark import Audio2landmark_model

## Step 1: Basic setup for the animation

In [2]:
default_head_name = 'paint_boy'           # the image name (with no .jpg) to animate
ADD_NAIVE_EYE = True                 # whether add naive eye blink
CLOSE_INPUT_FACE_MOUTH = False       # if your image has an opened mouth, put this as True, else False
AMP_LIP_SHAPE_X = 2.                 # amplify the lip motion in horizontal direction
AMP_LIP_SHAPE_Y = 2.                 # amplify the lip motion in vertical direction
AMP_HEAD_POSE_MOTION = 0.7           # amplify the head pose motion (usually smaller than 1.0, put it to 0. for a static head pose)

Default hyper-parameters for the model.

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--jpg', type=str, default='{}.jpg'.format(default_head_name))
parser.add_argument('--close_input_face_mouth', default=CLOSE_INPUT_FACE_MOUTH, action='store_true')

parser.add_argument('--load_AUTOVC_name', type=str, default='examples/ckpt/ckpt_autovc.pth')
parser.add_argument('--load_a2l_G_name', type=str, default='examples/ckpt/ckpt_speaker_branch.pth')
parser.add_argument('--load_a2l_C_name', type=str, default='examples/ckpt/ckpt_content_branch.pth') #ckpt_audio2landmark_c.pth')
parser.add_argument('--load_G_name', type=str, default='examples/ckpt/ckpt_116_i2i_comb.pth') #ckpt_image2image.pth') #ckpt_i2i_finetune_150.pth') #c

parser.add_argument('--amp_lip_x', type=float, default=AMP_LIP_SHAPE_X)
parser.add_argument('--amp_lip_y', type=float, default=AMP_LIP_SHAPE_Y)
parser.add_argument('--amp_pos', type=float, default=AMP_HEAD_POSE_MOTION)
parser.add_argument('--reuse_train_emb_list', type=str, nargs='+', default=[]) #  ['iWeklsXc0H8']) #['45hn7-LXDX8']) #['E_kmpT-EfOg']) #'iWeklsXc0H8', '29k8RtSUjE0', '45hn7-LXDX8',
parser.add_argument('--add_audio_in', default=False, action='store_true')
parser.add_argument('--comb_fan_awing', default=False, action='store_true')
parser.add_argument('--output_folder', type=str, default='examples')

parser.add_argument('--test_end2end', default=True, action='store_true')
parser.add_argument('--dump_dir', type=str, default='', help='')
parser.add_argument('--pos_dim', default=7, type=int)
parser.add_argument('--use_prior_net', default=True, action='store_true')
parser.add_argument('--transformer_d_model', default=32, type=int)
parser.add_argument('--transformer_N', default=2, type=int)
parser.add_argument('--transformer_heads', default=2, type=int)
parser.add_argument('--spk_emb_enc_size', default=16, type=int)
parser.add_argument('--init_content_encoder', type=str, default='')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--reg_lr', type=float, default=1e-6, help='weight decay')
parser.add_argument('--write', default=False, action='store_true')
parser.add_argument('--segment_batch_size', type=int, default=1, help='batch size')
parser.add_argument('--emb_coef', default=3.0, type=float)
parser.add_argument('--lambda_laplacian_smooth_loss', default=1.0, type=float)
parser.add_argument('--use_11spk_only', default=False, action='store_true')
parser.add_argument('-f')

opt_parser, unknown = parser.parse_known_args()

## Step 2: load the image and detect its landmark

In [4]:
img =cv2.imread('AI-square.jpg')
img = cv2.resize(img, (256, 256))
predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType.THREE_D, device='cpu', flip_input=True)
shapes = predictor.get_landmarks(img)
if (not shapes or len(shapes) != 1):
    print('Cannot detect face landmarks. Exit.')
    exit(-1)
shape_3d = shapes[0]

if(opt_parser.close_input_face_mouth):
    util.close_input_face_mouth(shape_3d)

shape_3d, scale, shift = util.norm_input_face(shape_3d)

In [5]:
print(256 / (scale * 256), shift)

-97.5 [-131.  -110.5]


In [6]:
print(256 / (0.010256410256410256 * 256))

97.5


In [7]:
#read json file
import json
with open('landmarks_annie.json', 'r') as f:
    data = json.load(f)

data_fls = []
for k, v in data.items():
    data_fls.append(v)

print(len(data_fls))

1357


In [8]:
import numpy as np

fl = np.loadtxt("landmarks_small_mouth.txt").reshape((-1, 68,3))

fl[:, :, 0:2] = -fl[:, :, 0:2]
fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift

fl0 = fl[0, :, :]

fl0 = fl0.reshape(-1, 204)

fls = []
for fl1 in data_fls:
    fls.append(fl0 + fl1)

print(fls[0])

[[ 6.48250000e+01  1.10500000e+02 -1.82168450e+00  5.86153750e+01
   1.32527200e+02 -1.87465430e+00  6.20971000e+01  1.52511775e+02
  -1.80245510e+00  6.57874750e+01  1.68132250e+02 -1.56746070e+00
   7.12442000e+01  1.87141825e+02 -1.21938810e+00  8.30816750e+01
   2.03738275e+02 -5.87100100e-01  9.42464000e+01  2.14600750e+02
   4.48520560e-01  1.06849250e+02  2.23722850e+02  1.35351060e+00
   1.29641825e+02  2.29153600e+02  1.37948620e+00  1.53342750e+02
   2.24697850e+02  1.08266510e+00  1.68938225e+02  2.14600750e+02
  -4.07776440e-01  1.82725700e+02  2.02763275e+02 -2.15717200e+00
   1.93588175e+02  1.87141825e+02 -3.40222300e+00  1.98347150e+02
   1.68132250e+02 -4.12142570e+00  2.01827900e+02  1.51536775e+02
  -4.47277670e+00  2.05309625e+02  1.32527200e+02 -4.45581600e+00
   2.05100000e+02  1.10500000e+02 -4.50210140e+00  6.85278500e+01
   9.71308000e+01 -7.33965400e-01  7.86249500e+01  8.68000500e+01
  -9.61628000e-02  9.46646750e+01  8.77750500e+01  2.17981180e-01
   1.02602

In [9]:
fl = np.array(fls)

fl = fl.reshape((-1, 68, 3))

fl = util.add_naive_eye(fl)

fl = fl.reshape((-1, 204))
fl[:, :48 * 3] = savgol_filter(fl[:, :48 * 3], 30, 3, axis=0)
fl[:, 48*3:] = savgol_filter(fl[:, 48*3:], 5, 3, axis=0)
fl = fl.reshape((-1, 68, 3))

''' STEP 6: Imag2image translation '''
model = Image_translation_block(opt_parser, single_test=True)
with torch.no_grad():
    model.single_test(jpg=img, fls=fl, filename="landmarks_small_mouth.txt", prefix=opt_parser.jpg.split('.')[0])
    print('finish image2image gen')

[-0.0421875  -0.0265625  -0.01205357  0.00133929  0.01361607  0.02477679
  0.03482143  0.04375     0.0515625   0.05825893  0.06383929  0.06830357
  0.07165179  0.07388393  0.075       0.075       0.07388393  0.07165179
  0.06830357  0.06383929  0.05825893  0.0515625   0.04375     0.03482143
  0.02477679  0.01361607  0.00133929 -0.01205357 -0.0265625  -0.0421875 ]
[-0.08571429  0.34285714  0.48571429  0.34285714 -0.08571429]
Run on device cpu


OpenCV: FFMPEG: tag 0x67706a6d/'mjpg' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Time - only video: 273.6616806983948
Time - ffmpeg add audio: 274.04273676872253
finish image2image gen


examples/.wav: No such file or directory


In [10]:
deltas_fl_to_0 = []
for f in fl:

    # rever this:
    # fl[:, :, 0:2] = -fl[:, :, 0:2]
    # fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift
    
    f[:, :, 0:2] = -f[:, :, 0:2]
    f[:, :, 0:2] = f[:, :, 0:2] * scale + shift

    deltas_fl_to_0.append(f - fl0)

print(deltas_fl_to_0[0])

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
landmarks_list = [
    "fake_small_mouth/0.txt",
    "fake_small_mouth/1.txt",
    "fake_small_mouth/2.txt",
    "fake_small_mouth/3.txt",
    "fake_small_mouth/4.txt",
    "fake_small_mouth/5.txt",
    "fake_small_mouth/6.txt",
    "fake_small_mouth/7.txt",
    "fake_small_mouth/8.txt",
    "fake_small_mouth/9.txt",
    "fake_small_mouth/10.txt",
    "fake_small_mouth/11.txt",
    "fake_small_mouth/12.txt",
    "fake_small_mouth/13.txt",
    "fake_small_mouth/14.txt",
    "fake_small_mouth/15.txt",
    "fake_small_mouth/16.txt",
    "fake_small_mouth/17.txt",
    "fake_small_mouth/18.txt",
    "fake_small_mouth/19.txt",
    "fake_small_mouth/20.txt",
    "fake_small_mouth/21.txt",
]

data = [{"audio": 50, "ID": 0},

{"audio": 100, "ID": 12},

{"audio": 212.5, "ID": 4},

{"audio": 262.5, "ID": 14},

{"audio": 350, "ID": 8},

{"audio": 512.5, "ID": 4},

{"audio": 675, "ID": 0},

{"audio": 1437.5, "ID": 0},

{"audio": 1487.5, "ID": 11},

{"audio": 1662.5, "ID": 1},

{"audio": 1712.5, "ID": 21},

{"audio": 1775, "ID": 6},

{"audio": 1825, "ID": 3},

{"audio": 1862.5, "ID": 13},

{"audio": 1900, "ID": 18},

{"audio": 1962.5, "ID": 5},

{"audio": 2012.5, "ID": 13},

{"audio": 2075, "ID": 19},

{"audio": 2112.5, "ID": 16},

{"audio": 2150, "ID": 4},

{"audio": 2200, "ID": 1},

{"audio": 2250, "ID": 14},

{"audio": 2300, "ID": 18},

{"audio": 2400, "ID": 13},

{"audio": 2462.5, "ID": 4},

{"audio": 2612.5, "ID": 19},

{"audio": 2687.5, "ID": 19},

{"audio": 2837.5, "ID": 0},

{"audio": 3650, "ID": 0},

{"audio": 3700, "ID": 11},

{"audio": 3875, "ID": 1},

{"audio": 3925, "ID": 21},

{"audio": 3987.5, "ID": 12},

{"audio": 4062.5, "ID": 6},

{"audio": 4137.5, "ID": 13},

{"audio": 4187.5, "ID": 19},

{"audio": 4225, "ID": 4},

{"audio": 4262.5, "ID": 12},

{"audio": 4350, "ID": 4},

{"audio": 4387.5, "ID": 14},

{"audio": 4450, "ID": 21},

{"audio": 4500, "ID": 6},

{"audio": 4600, "ID": 7},

{"audio": 4837, "ID": 0},

{"audio": 5650, "ID": 0},

{"audio": 5700, "ID": 11},

{"audio": 5825, "ID": 19},

{"audio": 5950, "ID": 8},

{"audio": 5975, "ID": 4},

{"audio": 6012.5, "ID": 17},

{"audio": 6062.5, "ID": 1},

{"audio": 6112.5, "ID": 19},

{"audio": 6162.5, "ID": 11},

{"audio": 6225, "ID": 19},

{"audio": 6325, "ID": 6},

{"audio": 6400, "ID": 19},

{"audio": 6462.5, "ID": 15},

{"audio": 6537.5, "ID": 1},

{"audio": 6587.5, "ID": 21},

{"audio": 6637.5, "ID": 6},

{"audio": 6687.5, "ID": 21},

{"audio": 6737.5, "ID": 21},

{"audio": 6800, "ID": 13},

{"audio": 6862.5, "ID": 7},

{"audio": 6912.5, "ID": 18},

{"audio": 6950, "ID": 21},

{"audio": 7000, "ID": 1},

{"audio": 7087.5, "ID": 19},

{"audio": 7150, "ID": 19},

{"audio": 7200, "ID": 15},

{"audio": 7375, "ID": 0}
]

from moviepy.editor import VideoFileClip, ImageSequenceClip
from moviepy.editor import *

frames = []

# List to hold the video clips
clips = []

#create a list of all the frames (30 fps) in 7500 ms
num_frames = int(30 * 7.5)
frames = []

list_landmarks = []
for i in range(num_frames):

    # get the viseme id for the specific frame based on the time of the data
    viseme_id = 0
    for data_point in data:
        if data_point["audio"] > i * 1000 / 30:
            viseme_id = data_point["ID"]
            break
    
    # get the landmark file for the specific viseme
    landmark_file = landmarks_list[viseme_id]
    landmark_data = np.loadtxt(landmark_file).tolist()
    
    list_landmarks.append(landmark_data)

# write to a txt file (one line per frame)
with open("landmarks_small_mouth.txt", "w") as f:
    for landmark in list_landmarks:
        for point in landmark:
            for p in point:
                f.write(str(p) + " ")
        f.write("\n")

# # Concatenate all the clips
# final_video = concatenate_videoclips(clips, method="compose")

# # Export the video
# final_video.write_videofile("final_output_video_2.mp4", fps=24)

In [ ]:
# for id in range(0, 22):
#     img =cv2.imread('fake/'+ str(id) + '.png')
#     predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType.THREE_D, device='cpu', flip_input=True)
#     shapes = predictor.get_landmarks(img)
#     if (not shapes or len(shapes) != 1):
#         print('Cannot detect face landmarks. Exit.')
#         exit(-1)
#     shape_3d = shapes[0]

#     util.close_input_face_mouth(shape_3d)

#     # make the mouth a little bit more open
#     # shape_3d[48:60, 1] -= 0.2
#     # shape_3d[48:60, 2] += 0.2

#     shape_3d[48 * 3::3] *= 6  # mouth x
#     shape_3d[48 * 3 + 1::3] *= 6  # mouth y

#     shape_3d, scale, shift = util.norm_input_face(shape_3d)

#     shape_3d = shape_3d.reshape(-1, 204)[0]

#     # write to a txt file in one line with 5 digits after the decimal point
#     np.savetxt('fake/'+ str(id) + '.txt', shape_3d, fmt='%.5f', newline=' ')

## (Optional) Simple manual adjustment to landmarks in case FAN is not accurate, e.g.
- slimmer lips
- wider eyes
- wider mouth

In [ ]:
shape_3d[48:, 0] = (shape_3d[48:, 0] - np.mean(shape_3d[48:, 0])) * 1.05 + np.mean(shape_3d[48:, 0]) # wider lips
shape_3d[49:54, 1] += 0.           # thinner upper lip
shape_3d[55:60, 1] -= 1.           # thinner lower lip
shape_3d[[37,38,43,44], 1] -=2.    # larger eyes
shape_3d[[40,41,46,47], 1] +=2.    # larger eyes

Normalize face as input to audio branch

In [ ]:
shape_3d, scale, shift = util.norm_input_face(shape_3d)

## Step 3: Generate input data for inference based on uploaded audio `MakeItTalk/examples/*.wav`

In [ ]:
!ffmpeg

In [ ]:
au_data = []
au_emb = []
ains = glob.glob1('examples', '*.wav')
ains = [item for item in ains if item is not 'tmp.wav']
ains.sort()
for ain in ains:
    os.system('/opt/homebrew/bin/ffmpeg -y -loglevel error -i examples/{} -ar 16000 examples/tmp.wav'.format(ain))
    shutil.copyfile('examples/tmp.wav', 'examples/{}'.format(ain))

    # au embedding
    from thirdparty.resemblyer_util.speaker_emb import get_spk_emb
    me, ae = get_spk_emb('examples/{}'.format(ain))
    au_emb.append(me.reshape(-1))

    print('Processing audio file', ain)
    c = AutoVC_mel_Convertor('examples')

    au_data_i = c.convert_single_wav_to_autovc_input(audio_filename=os.path.join('examples', ain),
           autovc_model_path=opt_parser.load_AUTOVC_name)
    au_data += au_data_i
if(os.path.isfile('examples/tmp.wav')):
    os.remove('examples/tmp.wav')

# landmark fake placeholder
fl_data = []
rot_tran, rot_quat, anchor_t_shape = [], [], []
for au, info in au_data:
    au_length = au.shape[0]
    fl = np.zeros(shape=(au_length, 68 * 3))
    fl_data.append((fl, info))
    rot_tran.append(np.zeros(shape=(au_length, 3, 4)))
    rot_quat.append(np.zeros(shape=(au_length, 4)))
    anchor_t_shape.append(np.zeros(shape=(au_length, 68 * 3)))

if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_fl.pickle'))
if(os.path.exists(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_fl_interp.pickle'))
if(os.path.exists(os.path.join('examples', 'dump', 'random_val_au.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_au.pickle'))
if (os.path.exists(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))):
    os.remove(os.path.join('examples', 'dump', 'random_val_gaze.pickle'))

with open(os.path.join('examples', 'dump', 'random_val_fl.pickle'), 'wb') as fp:
    pickle.dump(fl_data, fp)
with open(os.path.join('examples', 'dump', 'random_val_au.pickle'), 'wb') as fp:
    pickle.dump(au_data, fp)
with open(os.path.join('examples', 'dump', 'random_val_gaze.pickle'), 'wb') as fp:
    gaze = {'rot_trans':rot_tran, 'rot_quat':rot_quat, 'anchor_t_shape':anchor_t_shape}
    pickle.dump(gaze, fp)

## Step 4: Audio-to-Landmarks prediction

In [ ]:
!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

In [ ]:
!pwd
model = Audio2landmark_model(opt_parser, jpg_shape=shape_3d)
if(len(opt_parser.reuse_train_emb_list) == 0):
    model.test(au_emb=au_emb)
else:
    model.test(au_emb=None)

In [ ]:
def vis_landmark_on_img(img, shape, linewidth=2):
    '''
    Visualize landmark on images.
    '''

    def draw_curve(idx_list, color=(0, 255, 0), loop=False, lineWidth=linewidth):
        for i in idx_list:
            cv2.line(img, (shape[i, 0], shape[i, 1]), (shape[i + 1, 0], shape[i + 1, 1]), color, lineWidth)
        if (loop):
            cv2.line(img, (shape[idx_list[0], 0], shape[idx_list[0], 1]),
                     (shape[idx_list[-1] + 1, 0], shape[idx_list[-1] + 1, 1]), color, lineWidth)

    draw_curve(list(range(0, 16)), color=(255, 144, 25))  # jaw
    draw_curve(list(range(17, 21)), color=(50, 205, 50))  # eye brow
    draw_curve(list(range(22, 26)), color=(50, 205, 50))
    draw_curve(list(range(27, 35)), color=(208, 224, 63))  # nose
    draw_curve(list(range(36, 41)), loop=True, color=(71, 99, 255))  # eyes
    draw_curve(list(range(42, 47)), loop=True, color=(71, 99, 255))
    draw_curve(list(range(48, 59)), loop=True, color=(238, 130, 238))  # mouth
    draw_curve(list(range(60, 67)), loop=True, color=(238, 130, 238))

    return img

img = cv2.imread('real/2.png')

fl = np.loadtxt("landmarks.txt").reshape((-1, 68,3))
fl[:, :, 0:2] = -fl[:, :, 0:2]
fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift

if (ADD_NAIVE_EYE):
    fl = util.add_naive_eye(fl)

# additional smooth
fl = fl.reshape((-1, 204))
fl[:, :48 * 3] = savgol_filter(fl[:, :48 * 3], 15, 3, axis=0)
fl[:, 48*3:] = savgol_filter(fl[:, 48*3:], 5, 3, axis=0)
fl = fl.reshape((-1, 68, 3))

fl = fl.astype(int)
img_fl = np.ones(shape=(256, 256, 3)) * 255
img_fl = img_fl.astype(int)
img_fl = vis_landmark_on_img(img_fl, fl)


In [ ]:
import numpy as np

fl = np.loadtxt("landmarks_small_mouth.txt").reshape((-1, 68,3))

fl[:, :, 0:2] = -fl[:, :, 0:2]
fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift

# if (ADD_NAIVE_EYE):
#     fl = util.add_naive_eye(fl)

window_size = 10

# Iterate over each frame
for i in range(len(fl_filtered)):
    # Determine the start and end indices of the window
    if i < window_size:
        continue

    # Extract the subset for filtering
    subset = fl_filtered[i-10:i]

    # Apply Savitzky-Golay filter to the subset
    subset[:, :48 * 3] = savgol_filter(subset[:, :48 * 3], 10, 3, axis=0)
    subset[:, 48*3:] = savgol_filter(subset[:, 48*3:], 10, 3, axis=0)

    fl_filtered[i] = subset[i]

# Reshape back to the original shape
fl = fl_filtered.reshape((-1, 68, 3))

''' STEP 6: Imag2image translation '''
model = Image_translation_block(opt_parser, single_test=True)
with torch.no_grad():
    model.single_test(jpg=img, fls=fl, filename="landmarks.txt", prefix=opt_parser.jpg.split('.')[0])
    print('finish image2image gen')

## Step 5: Natural face animation via Image-to-image translation 

In [ ]:
# fls = glob.glob1('examples', 'pred_fls_*.txt')
# fls.sort()

# for i in range(0,len(fls)):
#     fl = np.loadtxt(os.path.join('examples', fls[i])).reshape((-1, 68,3))
#     print(fl.shape)
#     fl[:, :, 0:2] = -fl[:, :, 0:2]
#     fl[:, :, 0:2] = fl[:, :, 0:2] / scale - shift

#     if (ADD_NAIVE_EYE):
#         fl = util.add_naive_eye(fl)

#     # additional smooth
#     fl = fl.reshape((-1, 204))
#     fl[:, :48 * 3] = savgol_filter(fl[:, :48 * 3], 15, 3, axis=0)
#     fl[:, 48*3:] = savgol_filter(fl[:, 48*3:], 5, 3, axis=0)
#     fl = fl.reshape((-1, 68, 3))

#     ''' STEP 6: Imag2image translation '''
#     model = Image_translation_block(opt_parser, single_test=True)
#     with torch.no_grad():
#         model.single_test(jpg=img, fls=fl, filename=fls[i], prefix=opt_parser.jpg.split('.')[0])
#         print('finish image2image gen')

## Visualize your animation!

In [ ]:
from IPython.display import HTML
from base64 import b64encode

for ain in ains:
  OUTPUT_MP4_NAME = '{}_pred_fls_{}_audio_embed.mp4'.format(
    opt_parser.jpg.split('.')[0],
    ain.split('.')[0]
    )
  mp4 = open('examples/{}'.format(OUTPUT_MP4_NAME),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

  print('Display animation: examples/{}'.format(OUTPUT_MP4_NAME))
  display(HTML("""
  <video width=600 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))